<a href="https://colab.research.google.com/github/arthurst38/deep_learning/blob/main/G%C3%A9n%C3%A9ration_de_texte_RNN_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modèle de langage et génération de séquence

Ce TP a pour but de vous familiariser avec le concept de modèle de langage et de génération de séquence.

À partir d'un corpus de textes écrits par Voltaire, nous allons apprendre un modèle récurrent basé sur les séquences de caractères.

## Vérification de l'utilisation de GPU

Allez dans le menu `Exécution > Modifier le type d'execution` et vérifiez que l'on est bien en Python 3 et que l'accélérateur matériel est configuré sur « GPU ».

In [ ]:
!nvidia-smi

## Récupération des données

Les différents texte de Voltaire qui constituent le corpus ont été obtenus à partir de gutenberg.org. Les headers, footers ainsi que les préfaces ont été préalablement enlevés : on ne voudrait pas que notre modèle de langage apprenne à écrire les disclaimers de gutenberg.org ou les préface de l'éditeur.

On utilisera uniquement le fichier `dataset-voltaire/voltaire_clean.txt`.

In [ ]:
!rm -rf dataset-voltaire
!git clone https://github.com/nzmonzmp/dataset-voltaire.git
print("─" * 50)
!ls -l dataset-voltaire/
print("─" * 50)
!cat dataset-voltaire/voltaire_clean.txt


In [ ]:
# Utilisez cette cellule pour explorer un peu les données.
!wc dataset-voltaire/voltaire_clean.txt

In [ ]:
# Décommentez pour télécharger le fichier
# import google.colab
# google.colab.files.download('dataset-voltaire/concat_voltaire.txt')

## Import de TensorFlow et des autres librairies nécessaires

In [ ]:
import numpy
import pathlib
import typing

import tensorflow as tf
import tensorflow.keras as keras

## Chargement des données et extraction du vocabulaire

- *Chargez le texte du ficher en miniscules dans la variable `text`*
- *Extrayez tous les caractères utilisés dans `text` dans la liste `index_to_char`, triés par ordre alphabétique. Cette liste sert pour passer de la représentation numérique d'un caractère au caractère lui-même*
- *Construisez un dictionnaire `char_to_index` qui va permettre de passer d'un caractère à sa représentation numérique*

In [ ]:
text = "toto"
index_to_char = ["o", "t"]
char_to_index = {"o": 0, "t": 1}

### Solution

In [ ]:
text = pathlib.Path("dataset-voltaire/voltaire_clean.txt").read_text().lower()
print(f"{text[:100]}…")
print(f"Nombre de caractères : {len(text)}")

In [ ]:
index_to_char = sorted(set(text))
print(index_to_char)
print(f"Taille du dictionnaire : {len(index_to_char)}")

In [ ]:
char_to_index = {c: i for i, c in enumerate(index_to_char)}
print(char_to_index)

## Prétraitements

Découpage du texte en séquences de 30 caractères, tous les 3 caractères. Les cibles seront les 31èmes caractères

In [ ]:
maxlen = 30
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print(f"Nombre de séquences : {len(sentences)}")

In [ ]:
print(sentences[0])

## Création des tableaux d'entrée et de sortie

*Créez les variables `X` et `y` qui contiennent les données d'entrée et de sortie encodées.*

In [ ]:
# Votre code ici

### Solution

In [ ]:
def encode(sentence: str) -> numpy.ndarray:
  return numpy.array([char_to_index[char] for char in sentence])


X = numpy.vstack(map(encode, sentences))
y = numpy.array(encode(next_chars))

print(X.shape, y.shape)

## Préparation du modèle

*Essayez différents modèles ([`LSTM`](https://keras.io/api/layers/recurrent_layers/lstm/), [`GRU`](https://keras.io/api/layers/recurrent_layers/gru/), [`SimpleRNN`](https://keras.io/api/layers/recurrent_layers/simple_rnn/)) de différentes tailles.*

*Laissez quelques itérations à l'algorithme avant d'essayer une nouvelle configuration, ou utilisez [Keras Tuner](https://keras-team.github.io/keras-tuner/) pour le faire automatiquement.*

L'objectif étant évidemment de minimiser la loss : Plus elle est basse, plus le modèle est proche du modèle de langage de Voltaire.


In [ ]:
def build_model(embedding_dim: int = 8,
                lstm_hidden_dim: int = 128,
                learning_rate: float = 1e-3
               ) -> keras.models.Model:
  model = keras.models.Sequential()
  model.add(keras.layers.Embedding(len(index_to_char), embedding_dim))
  model.add(keras.layers.LSTM(lstm_hidden_dim))
  model.add(keras.layers.Dense(len(index_to_char), activation="softmax"))
  optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
  model.compile(loss="sparse_categorical_crossentropy",
                optimizer=optimizer,
                metrics=["accuracy"])
  return model

## Génération de texte

On génère dans le code qui suit les caractères un à un, en décalant progressivement l'entrée donnée pour qu'elle fasse toujours `maxlen` caractères.

`tf.random.categorical` permet d'échantillonner depuis une distribution de probabilité, c'est donc notre outil principal pour exploiter la sortie du réseau : le softmax définit en effet une telle distribution.

In [ ]:
def sample(n_steps: int = 200):
  start_index = numpy.random.randint(0, len(text) - maxlen - 1)
  sentence = text[start_index:start_index + maxlen]
  print("─" * 50)
  print(f"Génération à partir de : « {sentence} »")
  print("─" * 50)
  print(sentence, end="")

  for _ in range(n_steps):
    word_probas = model.predict(encode(sentence)[None, :], verbose=0)
    next_index = tf.random.categorical(tf.math.log(word_probas), 1)[0][0]
    next_char = index_to_char[next_index]
    sentence = sentence[1:] + next_char
    print(next_char, end="")
  print()

## Apprentissage

Plutôt que de générer toutes les `EPOCHS` epochs comme on le fait là à l'aide d'une boucle extérieure, on aurait aussi pu utiliser un [callback Keras](https://keras.io/api/callbacks/lambda_callback/), mais on ne souhaite pas générer à toutes les epochs et cette façon de faire est donc dans ce cas plus pratique.

In [ ]:
EPOCHS = 10
model = build_model()
for i in range(10):
  model.fit(X, y,
            batch_size=4096,
            epochs=EPOCHS)
  print()
  print("─" * 50)
  print(f"Après {(i + 1) * EPOCHS} epochs :")
  sample()

## Une solution avec plusieurs couches de LSTM

In [ ]:
def build_deep_model(embedding_dim: int = 8,
                     lstm_hidden_dim: int = 128,
                     learning_rate: float = 1e-3
                    ) -> keras.models.Model:
  model = keras.models.Sequential()
  model.add(keras.layers.Embedding(len(index_to_char), embedding_dim))
  model.add(keras.layers.LSTM(lstm_hidden_dim, return_sequences=True))
  model.add(keras.layers.LSTM(lstm_hidden_dim))
  model.add(keras.layers.Dense(len(index_to_char), activation="softmax"))
  optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
  model.compile(loss="sparse_categorical_crossentropy",
                optimizer=optimizer,
                metrics=["accuracy"])
  return model

In [ ]:
EPOCHS = 10
model = build_deep_model()
for i in range(10):
  model.fit(X, y,
            batch_size=4096,
            epochs=EPOCHS)
  print()
  print("─" * 50)
  print(f"Après {(i + 1) * EPOCHS} epochs :")
  sample()